# 🚀 Lean Infinity Dual Hybrid v1 - Google Colab

This notebook runs the unified **Lean Infinity Dual Hybrid v1** benchmarks on Google Colab with GPU support.

**Controllers available (LM mode):**
- `transformer` - Standard Transformer decoder (baseline)
- `mamba` - Mamba backbone (Mamba2 CUDA if installed, GRU fallback)
- `mamba_dualmem` - Mamba + DualTierMiras parametric memory

**Benchmarks (LM mode):**
- Synthetic sequence tasks: `copy_memory`, `assoc_recall`, `selective_copy`, `induction_head`

**Entrypoint:**
- `python -m trans_mamba_core.unified_runner --mode lm ...`

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Upload your zip file
from google.colab import files

print("Upload Trans_MAMBA-lean-infinity-dual-hybrid-v1.zip:")
uploaded = files.upload()

In [ ]:
# Unzip
!unzip -o Trans_MAMBA-lean-infinity-dual-hybrid-v1.zip -d /content
%cd /content/Trans_MAMBA-lean-infinity-dual-hybrid-v1
!ls -la

In [ ]:
# Run unified setup script (installs PyTorch, dependencies, and optionally Mamba CUDA)\n# This handles everything automatically!\n!python setup_colab.py --install-all

In [ ]:
# Optional: Install Mamba2 from LOCAL SOURCE (takes ~5 min to compile)\n# This gives better performance than the PyPI version\n# Uncomment to enable:\n\n# !python setup_colab.py --install-mamba-source

## 2. (Optional) Install Mamba2 CUDA Kernels

This enables real Mamba2 SSM layers instead of GRU fallback. Skip if you want faster setup.

In [ ]:
# Optional: Install real Mamba2 from LOCAL SOURCE (takes ~5 min to compile)
# Uncomment to enable:

# %cd /content/Trans_MAMBA-lean-infinity-dual-hybrid-v1/external/mamba_ssm
# !pip install -e . --quiet
# %cd /content/Trans_MAMBA-lean-infinity-dual-hybrid-v1

## 3. Sanity Check

Verify all components work correctly.

In [ ]:
!python -m trans_mamba_core.unified_runner \
    --mode lm --task copy_memory --controller mamba_dualmem \
    --seq_len 64 --delay 16 --epochs 1 --batch_size 64

## 4. Synthetic Benchmarks

### 4.1 Copy Memory Task

Tests ability to copy a sequence after a delay period.

In [ ]:
# Transformer baseline
!python -m trans_mamba_core.unified_runner \
    --mode lm --task copy_memory \
    --controller transformer \
    --seq_len 100 --delay 40 \
    --epochs 20 --batch_size 64

In [ ]:
# Mamba baseline
!python -m trans_mamba_core.unified_runner \
    --mode lm --task copy_memory \
    --controller mamba \
    --seq_len 100 --delay 40 \
    --epochs 20 --batch_size 64

In [ ]:
# Mamba + DualTierMiras
!python -m trans_mamba_core.unified_runner \
    --mode lm --task copy_memory \
    --controller mamba_dualmem \
    --seq_len 100 --delay 40 \
    --epochs 20 --batch_size 64

In [ ]:
# Note: The v1 unified runner does not expose an `ot_agent` controller in LM mode.
# Use `mamba_dualmem` as the long-context baseline for these tasks.

### 4.2 Associative Recall Task

Tests content-addressable memory retrieval.

In [ ]:
# Compare all controllers on associative recall
for controller in ["transformer", "mamba", "mamba_dualmem"]:
    print(f"\n{'='*60}")
    print(f"Controller: {controller}")
    print("=" * 60)
    !python -m trans_mamba_core.unified_runner \
        --mode lm --task assoc_recall \
        --controller {controller} \
        --seq_len 30 --num_pairs 6 \
        --epochs 20 --batch_size 64

## 5. Notes on LM Benchmarks

The v1 core focuses on **synthetic sequence tasks** via `trans_mamba_core.unified_runner --mode lm`.

If you want to run a true text-corpus LM benchmark, we can extend the runner to accept `--data_path` again,
or add a separate script that wraps `trans_mamba_core.synthetic.lm_bench` against a tokenized dataset.

In [ ]:
# (Optional) Download a sample corpus (Shakespeare)
# This is not consumed by the v1 unified runner yet, but kept here for convenience.
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O /content/corpus.txt
!head -20 /content/corpus.txt
!wc -c /content/corpus.txt

In [ ]:
# Example: Induction Head synthetic task
!python -m trans_mamba_core.unified_runner \
    --mode lm --task induction_head \
    --controller transformer \
    --seq_len 128 \
    --epochs 10 --batch_size 64

In [ ]:
# Example: Selective Copy synthetic task
!python -m trans_mamba_core.unified_runner \
    --mode lm --task selective_copy \
    --controller mamba_dualmem \
    --seq_len 128 --delay 40 \
    --epochs 10 --batch_size 64

In [ ]:
# Example: RL mode quick run (CartPole)
!python -m trans_mamba_core.unified_runner \
    --mode rl --agent infinity --env cartpole \
    --num_updates 20 --log_interval 5

## 6. Full Comparison (All Controllers)

Run a comprehensive comparison and save logs.

In [ ]:
import json
from pathlib import Path

# Create logs directory
log_dir = Path("/content/logs")
log_dir.mkdir(parents=True, exist_ok=True)

controllers = ["transformer", "mamba", "mamba_dualmem"]
tasks = ["copy_memory", "assoc_recall"]

for task in tasks:
    for controller in controllers:
        print(f"\n{'='*60}")
        print(f"Task: {task} | Controller: {controller}")
        print("=" * 60)

        extra_args = ["--delay", "40"] if task == "copy_memory" else ["--num_pairs", "6"]
        seq_len = "100" if task == "copy_memory" else "30"

        !python -m trans_mamba_core.unified_runner \
            --mode lm --task {task} \
            --controller {controller} \
            --seq_len {seq_len} \
            --epochs 20 --batch_size 64 \
            --save_dir {str(log_dir)} \
            {" ".join(extra_args)}

In [ ]:
# View saved logs
!ls -la /content/logs/

In [ ]:
import json
from pathlib import Path

log_dir = Path("/content/logs")
results = []

for path in sorted(log_dir.glob("*.json")):
    with open(path) as f:
        payload = json.load(f)

    results.append(
        {
            "task": payload.get("task"),
            "controller": payload.get("controller"),
            "final_accuracy": payload.get("results", {}).get("final_accuracy"),
            "best_accuracy": payload.get("results", {}).get("best_accuracy"),
        }
    )

print(f"{'Task':<15} {'Controller':<15} {'Final Acc':<10} {'Best Acc':<10}")
print("-" * 55)
for r in results:
    final_acc = r["final_accuracy"]
    best_acc = r["best_accuracy"]
    final_s = f"{final_acc:.4f}" if isinstance(final_acc, (int, float)) else "N/A"
    best_s = f"{best_acc:.4f}" if isinstance(best_acc, (int, float)) else "N/A"
    print(f"{r['task']:<15} {r['controller']:<15} {final_s:<10} {best_s:<10}")

## 7. Download Results

In [ ]:
# Zip and download logs
!zip -r /content/benchmark_logs.zip /content/logs

from google.colab import files
files.download('/content/benchmark_logs.zip')

## 8. Custom Experiments

Modify parameters below for your own experiments.

In [ ]:
# Custom experiment parameters
CONTROLLER = "mamba_dualmem"  # transformer, mamba, mamba_dualmem
TASK = "copy_memory"          # copy_memory, assoc_recall, selective_copy, induction_head
SEQ_LEN = 200                 # Sequence length
DELAY = 80                    # Delay for copy_memory
EPOCHS = 30
BATCH_SIZE = 64

extra_args = "--delay {}".format(DELAY) if TASK == "copy_memory" else ""

!python -m trans_mamba_core.unified_runner \
    --mode lm --task {TASK} \
    --controller {CONTROLLER} \
    --seq_len {SEQ_LEN} {extra_args} \
    --epochs {EPOCHS} --batch_size {BATCH_SIZE} \
    --save_dir /content/logs